In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

2022-12-14 03:52:07.762060: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 03:52:08.047787: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-14 03:52:08.093179: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-14 03:52:08.093235: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
with open("billion-word-imputation/train_v2.txt/train_v2x", mode='r') as file:
    data = file.read()
    data = data.splitlines()

In [5]:
data[:5]

['The U.S. Centers for Disease Control and Prevention initially advised school systems to close if outbreaks occurred , then reversed itself , saying the apparent mildness of the virus meant most schools and day care centers should stay open , even if they had confirmed cases of swine flu .',
 'When Ms. Winfrey invited Suzanne Somers to share her controversial views about bio-identical hormone treatment on her syndicated show in 2009 , it won Ms. Winfrey a rare dollop of unflattering press , including a Newsweek cover story titled " Crazy Talk : Oprah , Wacky Cures & You . "',
 'Elk calling -- a skill that hunters perfected long ago to lure game with the promise of a little romance -- is now its own sport .',
 "Don 't !",
 'Fish , ranked 98th in the world , fired 22 aces en route to a 6-3 , 6-7 ( 5 / 7 ) , 7-6 ( 7 / 4 ) win over seventh-seeded Argentinian David Nalbandian .']

In [6]:
inputs = tokenizer(data, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [7]:
inputs

{'input_ids': tensor([[  101,  1996,  1057,  ...,     0,     0,     0],
        [  101,  2043,  5796,  ...,     0,     0,     0],
        [  101, 18995,  4214,  ...,     0,     0,     0],
        ...,
        [  101,  2899,  1011,  ...,     0,     0,     0],
        [  101,  5439,  2053,  ...,     0,     0,     0],
        [  101,  2002,  2187,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [8]:
inputs['labels'] = inputs.input_ids.clone().detach()
inputs

{'input_ids': tensor([[  101,  1996,  1057,  ...,     0,     0,     0],
        [  101,  2043,  5796,  ...,     0,     0,     0],
        [  101, 18995,  4214,  ...,     0,     0,     0],
        ...,
        [  101,  2899,  1011,  ...,     0,     0,     0],
        [  101,  5439,  2053,  ...,     0,     0,     0],
        [  101,  2002,  2187,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  1996,  1057,  ...,     0,     0,     0],
        [  101,  2043,  5796,  ...,     0,     0,     0],
        [  101, 18995, 

In [9]:
rand = torch.rand(inputs.input_ids.shape)

In [10]:
rand.shape

torch.Size([1000, 512])

In [11]:
rand

tensor([[0.9572, 0.2344, 0.9156,  ..., 0.5105, 0.2983, 0.2080],
        [0.8600, 0.1095, 0.6314,  ..., 0.9516, 0.0672, 0.1214],
        [0.7147, 0.2251, 0.0664,  ..., 0.8304, 0.4954, 0.3854],
        ...,
        [0.3828, 0.7831, 0.0236,  ..., 0.1263, 0.7813, 0.3250],
        [0.9009, 0.5596, 0.3669,  ..., 0.6042, 0.5270, 0.4128],
        [0.1230, 0.8055, 0.5569,  ..., 0.4387, 0.7463, 0.8004]])

In [12]:
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        ...,
        [False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [13]:
selection = list()

for i in range(mask_arr.shape[0]):
    selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

In [14]:
selection[:5]

[[4, 15, 18, 19, 23, 25, 33, 36, 39, 41],
 [1, 13, 20, 22, 52, 57],
 [2, 4, 12, 19, 26],
 [],
 [13, 17, 27, 34, 43]]

In [15]:
for i in range(mask_arr.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [16]:
inputs.input_ids

tensor([[  101,  1996,  1057,  ...,     0,     0,     0],
        [  101,   103,  5796,  ...,     0,     0,     0],
        [  101, 18995,   103,  ...,     0,     0,     0],
        ...,
        [  101,  2899,   103,  ...,     0,     0,     0],
        [  101,  5439,  2053,  ...,     0,     0,     0],
        [  101,  2002,  2187,  ...,     0,     0,     0]])

In [17]:
class BillionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        
    def __getitem__(self, index):
        return {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings.input_ids)

In [18]:
dataset = BillionDataset(inputs)

In [19]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [20]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [21]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [22]:
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [23]:
from transformers import AdamW

In [24]:
optim = AdamW(model.parameters(), lr=1e-5)

/home/lemaurkydd/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
from tqdm import tqdm

In [ ]:
epochs = 2

for epoch in range(epochs):
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()
        
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|                                                                                            | 0/63 [00:00<?, ?it/s]/tmp/ipykernel_1588/1843686157.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
